<b><p style="text-align:center;"> PROJET FRAGRAI </p></b>

<b><p style="color:blue;size=600">Importation </p></b>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import Counter
import difflib

In [2]:
difflib.SequenceMatcher(None, "arb","rbt").ratio()>0

True

<b><p style="color:blue;size=600">Load Data </p></b>

In [3]:
#Transform excel to csv
read_file = pd.read_excel (r'DataScrap/dataset.xlsx')
read_file.to_csv (r'dataset.csv', index = None, header=True)

#
df=pd.read_csv("dataset.csv", sep=",")


In [4]:
len("CI 14700 – CI 19140 – CI 17200 (BROWN 1 REPLACEMENT")

51

<b><p style="color:blue;size=600">Data Cleaning </p></b>

In [14]:
liste_corve=["DIENTS ENTRANT DANS LA COMPOSITION DES PRODUITS DE NOTRE MARQUE SONT R&EACUTE"," AVANT D'UTILISER UN PRODUIT DE NOTRE MARQUE"," VOTRE UTILISATION PERSONNELLE","DA L’ORÉAL REGELMÄßIG NEUE WISSENSCHAFTLICHE ERKENNTNISSE IN DIE PRODUKTFORMULIERUNG EINFLIEßEN LÄSST"," KÖNNEN VON EINER REFERENZ VERSCHIEDENE VERSIONEN MIT UNTERSCHIEDLICHEN ZUSAMMENSETZUNGEN IM UMLAUF SEIN","TES INVIT&EACUTE"," PARFUM"," SIE GELTEN STETS NUR FÜR DAS JEWEILS BEZEICHNETE PRODUKT MIT DER DAZU ANGEGEBENEN PRODUKT-REFERENZNUMMER"," PARFUM (FRAGRANCE) AQUA (WATER) BENZYL SALICYLATE LINALOOL HEXYL CINNAMAL HYDROXYCITRONELLAL ALPHA-ISOMETHYL IONONE CITRONELLOL LIMONENE GERANIOL EUGENOL BENZYL BENZOATE BENZYL ALCOHOL CITRAL","DIENTS FIGURANT SUR SON EMBALLAGE AFIN DE VOUS ASSURER QUE LES INGR&EACUTE"]
def CleanMax(string):
    if ("AQUA" in string) or ("WATER" in string )or ("EAU" in string):
        return "WATER"
    if "PARFUM" in string or "FRAGRANCE" in string:
        liste_corve.append(string)
        return "PARFUM"
    if len(string)<3 or len(string)>60:
        liste_corve.append(string)
        print(string)
        return "NONE"
    else :
        return string
#Création d'un dictionnaire {ingrédient:moyennes des rating} et dictionnaire {ingrédient:nbr_occurence}
liste_with_all_ingredient=[]
dicte_of_rating={}
for i,j in zip(df["Ingr"],df["Rating"]):
    ingredient=i.split(",")
    for j_ingredient in ingredient:
        j_ingredient=CleanMax(j_ingredient)
        liste_with_all_ingredient.append(j_ingredient)
    for ii in i.split(","):
        if CleanMax(ii) in dicte_of_rating:
            dicte_of_rating[CleanMax(ii)].append(j)
        else:
            dicte_of_rating[CleanMax(ii)]=[j]
print(type(dicte_of_rating))
dicte_of_rating={i:np.around(np.mean(j),2) for (i,j) in dicte_of_rating.items()}
dicte_of_rating

  
  
  
 B
 B
 B
DIENTS FIGURANT SUR SON EMBALLAGE AFIN DE VOUS ASSURER QUE LES INGR&EACUTE
DIENTS FIGURANT SUR SON EMBALLAGE AFIN DE VOUS ASSURER QUE LES INGR&EACUTE
DIENTS FIGURANT SUR SON EMBALLAGE AFIN DE VOUS ASSURER QUE LES INGR&EACUTE
 ALPHA-ISOMETHYL IONONE – CITRAL – CITRONELLOL – COUMARIN – GERANIOL – LIMONENE – LINALOOL
 ALPHA-ISOMETHYL IONONE – CITRAL – CITRONELLOL – COUMARIN – GERANIOL – LIMONENE – LINALOOL
 ALPHA-ISOMETHYL IONONE – CITRAL – CITRONELLOL – COUMARIN – GERANIOL – LIMONENE – LINALOOL
 DISCLAIMER:L’ORÉAL STELLT IHNEN DIE NACHSTEHENDEN INHALTSSTOFFLISTEN AUF IHREN WUNSCH ZUR VERFÜGUNG
B
 DISCLAIMER:L’ORÉAL STELLT IHNEN DIE NACHSTEHENDEN INHALTSSTOFFLISTEN AUF IHREN WUNSCH ZUR VERFÜGUNG
 DISCLAIMER:L’ORÉAL STELLT IHNEN DIE NACHSTEHENDEN INHALTSSTOFFLISTEN AUF IHREN WUNSCH ZUR VERFÜGUNG
B
B
 AUF EINEM PRODUKT FINDET SICH DIESE JEWEILS AM ENDE DER INHALTSSTOFFLISTE
 AUF EINEM PRODUKT FINDET SICH DIESE JEWEILS AM ENDE DER INHALTSSTOFFLISTE
 AUF EINEM PRODUKT FINDET

{' RICINUS COMMUNIS (CASTOR) SEED OIL': 4.2,
 ' STEARETH-100': 3.84,
 ' GLYCERYL STEARATE': 4.13,
 ' LIMONENE': 3.54,
 ' 60730)': 3.79,
 ' CI 77163 (BISMUTH OXYCHLORIDE)': 3.98,
 ' 17200 (RED 33)': 3.7,
 ' BUTYL METHOXYDIBENZOYLMETHANE': 4.07,
 'GERANIOL': 4.27,
 ' B28669/1)': 3.88,
 ' SODIUM COCOAMPHOACETATE': 3.86,
 ' DIMETHICONE': 3.92,
 ' CI 42090 (FD&C BLUE NO': 4.01,
 ' BUTYLPHENYL METHYLPROPIONAL': 3.81,
 ' CETYL PEG PPG-10 1 DIMETHICONE': 4.09,
 '2-HEXANEDIOL': 3.83,
 ' CI 19140 / YELLOW 5': 4.12,
 ' ORANGE 4 (CI 15510)': 3.89,
 ' HYDROXYISOHEXYL 3-CYCLOHEXENE CARBOXALDEHYD': 4.42,
 ' POLYQUATERNIUM-7': 4.11,
 ' ALUMINUM CHLOROHYDRATE': 3.96,
 ' C251806/1)': 4.01,
 ' AGAR': 3.91,
 'WATER': 4.05,
 ' CITRIC ACID – BHT': 3.94,
 ' CI 42090 / BLUE 1': 3.75,
 'ETHYL ALCOHOL DENAT': 3.95,
 '#13380 ALCOHOL': 4.57,
 ' CI 42090 (BLUE 1)': 4.08,
 ' ALPHA-ISOMETHYL IONONE': 4.03,
 ' \nCINNAMAL': 4.05,
 ' SODIUM PHOSPHATE': 4.43,
 ' CI17200': 4.09,
 ' 4 (CI 14700)': 4.22,
 ' LYCIUM BARBARUM

In [15]:
liste_of_occ=Counter(liste_with_all_ingredient)
sorted(liste_of_occ.items(), key=lambda t: t[1])

[(' BEZYL SALICYLATE', 4),
 (' B172401/1)', 5),
 (' ETHYLHEXYL SALICYLATE', 5),
 (' CI 42090(BLUE 1)', 5),
 (' CI 60730 / EXT', 6),
 ('33 (CI 17200)', 6),
 (' TRIMETHYLSILOXYSILICATE', 6),
 (' BUTHYL METHOXYDIBENZOYLMETHANE', 6),
 (' PENTAERYTHRITYL TETREA-DI-T-BUTYL HYDROXYHYDROCINNAMATE', 6),
 (' COVABSORB', 7),
 (' CI 47005 (YELLOW 10)', 7),
 (' CI14700 (RED 4)', 7),
 ('INGREDIENTS  BUTYL ACETATE', 7),
 (' LINALOO', 7),
 ('REMENT MISES &AGRAVE', 7),
 (' EXT', 8),
 (' DIETHYLAMINO HYDROXYBENZOYL HEXYL BENZOATE –', 8),
 (' TRIHYDROXYSTEARIN', 8),
 (' LINALLOL', 8),
 (' RED 4 (CI4700)', 8),
 (' HYDROXYCITRONELLAL', 8),
 (' LINALOOL', 8),
 (' HELIANTHUS ANNUUS SEED OIL', 8),
 (' ACACIA SENEGAL GUM', 8),
 (' SODIUM OLEATE', 8),
 (' CITRAL', 8),
 (' HEXYL CINNAMAL', 8),
 ('5 (CI 19140)', 8),
 (' YELLOW 5 (CI 19140)', 8),
 (' 40)', 8),
 (' BUTYL METHOXYDIBENZOYLMENTHANE', 8),
 (' POTASSIUM SORBATE', 8),
 ('BUTYL METHOXYDIBENZOYLMETHANE', 8),
 (' ALCOHOL DENAT', 9),
 (' HYDROGENATED STYRENE

In [7]:
for (key,value) in dicte_of_rating.items():
    for occurence in liste_of_occ:
        if key==occurence[0]:
            dicte_of_rating[key]=(value,occurence[1])
dicte_of_rating

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#Cleaning all the ingredient ou la note est < 3.5 et occ < 8
for i in dicte_of_rating:
    if dicte_of_rating[i][0]<4 and dicte_of_rating[i][1]<20:
        liste_corve.append(i)
len(liste_corve),len(dicte_of_rating)

In [ ]:
liste_of_occ


In [ ]:
difflib.SequenceMatcher(None, "PARFUM ( FRAGRANCE)","WATER(AQUA/EAU)").ratio()

In [ ]:
plt.hist(np.array(liste_of_occ)[:,1],20)